In [1]:
import pandas as pd
import os

In [2]:
files = os.listdir("../cleaned")
print("Cleaned Files:\n",files)
files2 = os.listdir("../raw")
print("Raw Files:\n",files2)

Cleaned Files:
 ['EPL.csv', 'spainranking.csv', 'engranking.csv', 'Liga.csv', 'itranking.csv', 'model_data.csv', 'SerieA.csv']
Raw Files:
 ['players_20.csv', 'players_18.csv', 'players_19.csv', 'fifa21_male2.csv', 'players_17.csv', 'big_five_1995-2019.csv', 'players_16.csv', 'players_15.csv', 'team_stats.xlsx', 'teams_and_leagues.csv']


In [3]:
xl = pd.ExcelFile("../raw/team_stats.xlsx")
xl.sheet_names

['team_stats', 'epl', 'Sheet3']

In [4]:
liga = pd.read_excel("../raw/team_stats.xlsx",sheet_name="team_stats",header=0)
epl = pd.read_excel("../raw/team_stats.xlsx",sheet_name="epl",header=0)
serieA = pd.read_excel("../raw/team_stats.xlsx",sheet_name="Sheet3",header=0)

In [5]:
liga.rename({'Rank':'Ranking'},axis=1, inplace=True)
epl.rename({'Rank':'Ranking'},axis=1, inplace=True)
serieA.rename({'Rank':'Ranking'},axis=1, inplace=True)

In [6]:
transfers_it = pd.read_csv("../../transfers/cleaned/italy_joined.csv", dtype={'Ranking':"Int64"})
transfers_it["Country"] = 'Italy'
transfers_es = pd.read_csv("../../transfers/cleaned/spain_joined.csv",dtype={'Ranking':"Int64"})
transfers_es["Country"] = 'Spain'
transfers_en = pd.read_csv("../../transfers/cleaned/england_joined.csv",dtype={'Ranking':"Int64"})
transfers_en["Country"] = 'England'
transfer_df = pd.concat([transfers_it,transfers_es,transfers_en])
del transfer_df["Unnamed: 0"]
transfer_df.head()

,Year,Teams,Ranking,avg player value (EU),avg player wage,Transfer Spend (Euros),MA,Country
0,2019,Juventus,1,22287121.0,113636.0,201.150,194.28,Italy
1,2019,Inter Milan,2,14953666.0,53733.0,171.284,127.64,Italy
2,2019,SS Lazio,3,11658939.0,40455.0,41.078,36.29,Italy
3,2019,Atalanta,4,8712321.0,38393.0,51.130,49.78,Italy
4,2019,AS Roma,5,9323709.0,31516.0,102.690,110.81,Italy


In [7]:
transfer_df = transfer_df[["Year","Ranking","Transfer Spend (Euros)","MA","Country"]]
transfer_df.dropna(subset=["Ranking"],inplace=True)
transfer_df.head()

,Year,Ranking,Transfer Spend (Euros),MA,Country
0,2019,1,201.150,194.28,Italy
1,2019,2,171.284,127.64,Italy
2,2019,3,41.078,36.29,Italy
3,2019,4,51.130,49.78,Italy
4,2019,5,102.690,110.81,Italy


In [8]:
eng_df = pd.read_csv(f"../cleaned/{files[0]}",encoding="UTF-8")
final_eng = eng_df.merge(epl,on=['Ranking','Year'],how='outer')
final_eng = final_eng.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_eng['Goals+'].iloc[0:20] = final_eng['G+'].iloc[0:20]
final_eng['Goals-'].iloc[0:20] = final_eng['G-'].iloc[0:20]
final_eng['Points_x'].iloc[0:20] = final_eng['Points_y'].iloc[0:20]
final_eng['Teams'].iloc[-20:] = final_eng['Team'].iloc[-20:]
final_eng['Points_x'].iloc[-20:] = final_eng['Points_y'].iloc[-20:]
final_eng['Goals+'].iloc[-20:] = final_eng['G+'].iloc[-20:]
final_eng['Goals-'].iloc[-20:] = final_eng['G-'].iloc[-20:]
final_eng["Country"] = 'England'
final_eng = final_eng.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [9]:
es_df = pd.read_csv(f"../cleaned/{files[3]}",encoding="UTF-8")
final_es = es_df.merge(liga,on=['Ranking','Year'],how='outer')
final_es = final_es.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_es['Goals+'].iloc[0:20] = final_es['G+'].iloc[0:20]
final_es['Goals-'].iloc[0:20] = final_es['G-'].iloc[0:20]
final_es['Teams'].iloc[-20:] = final_es['Team'].iloc[-20:]
final_es['Points_x'].iloc[-20:] = final_es['Points_y'].iloc[-20:]
final_es['Goals+'].iloc[-20:] = final_es['G+'].iloc[-20:]
final_es['Goals-'].iloc[-20:] = final_es['G-'].iloc[-20:]
final_es["Country"] = 'Spain'
final_es = final_es.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [10]:
it_df = pd.read_csv(f"../cleaned/{files[6]}",encoding="UTF-8")
final_it = it_df.merge(serieA,on=['Ranking','Year'],how='outer')
final_it = final_it.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_it['Goals+'].iloc[0:20] = final_it['G+'].iloc[0:20]
final_it['Goals-'].iloc[0:20] = final_it['G-'].iloc[0:20]
final_it['Points_x'].iloc[0:20] = final_it['Points_y'].iloc[0:20]
final_it['Teams'].iloc[-20:] = final_it['Team'].iloc[-20:]
final_it['Points_x'].iloc[-20:] = final_it['Points_y'].iloc[-20:]
final_it['Goals+'].iloc[-20:] = final_it['G+'].iloc[-20:]
final_it['Goals-'].iloc[-20:] = final_it['G-'].iloc[-20:]
final_it["Country"] = 'Italy'
final_it = final_it.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [11]:
leagues_df = pd.concat([final_eng,final_it,final_es])
leagues_df = leagues_df.merge(transfer_df, on=["Year","Ranking","Country"],how='outer').drop_duplicates()
leagues_df.fillna(0,inplace=True)

In [39]:
#import the data for 2020
from selenium import webdriver
import time
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
url_eng = "https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/TeamStatistics/England-Premier-League-2020-2021"
url_es = "https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/8321/Stages/18851/TeamStatistics/Spain-LaLiga-2020-2021"
url_it = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/TeamStatistics/Italy-Serie-A-2020-2021"
urls = [url_eng,url_es,url_it]
dfs = []
for url in urls:
    driver.get(url)
    time.sleep(3)
    tbl = driver.find_element_by_xpath(
        '//table[@id="top-team-stats-summary-grid"]').get_attribute('outerHTML')
    df  = pd.read_html(tbl)[0]
    for i,d in df.iterrows():
        teams = d["Team"].split(" ",1)
        df["Team"][i] = teams[1]
    df = df[["Team","Shots pg", "Possession%","Pass%"]].rename({"Team":"Teams"}, axis=1)
    dfs.append(df)
df = pd.concat(dfs)
df2 = leagues_df.groupby("Teams").agg(
    {"Goals+":"mean","Goals-":"mean","MA":"mean"}).\
        sort_values("Goals+", ascending=False)
df_final = df.merge(df2,on=["Teams"], how="inner")
df_final.head()

,Teams,Shots pg,Possession%,Pass%,Goals+,Goals-,MA
0,Aston Villa,15.9,50.1,78.0,26.750000,42.000000,0.000000
1,Manchester City,15.6,59.8,88.3,57.857143,21.285714,134.934286
2,Liverpool,15.4,60.5,86.0,49.142857,22.857143,92.430000
3,Chelsea,13.7,56.9,86.1,46.285714,25.000000,115.608571
4,Tottenham,11.3,48.5,79.9,25.000000,12.000000,0.000000


In [53]:
def playersfunc2(filepath,year):
    players_df = pd.read_csv(filepath)
    players_df = players_df[['Club','OVA']]
    players_df = players_df.rename({"Club":"Teams",'OVA':"overall"},axis=1)
    f = {"overall":"mean"}
    players_df = players_df.groupby("Teams").agg(f)
    players_df["Year"]= int(year)
    players_df = players_df.reset_index()
    return cleanup(players_df)
def cleanup(df):
    df["Teams"] = df["Teams"].str.replace("RC Celta Vigo", "Celta Vigo")
    df["Teams"] = df["Teams"].str.replace("Real Sporting de Gijón", "Sporting Gijón")
    df["Teams"] = df["Teams"].str.replace("Deportivo de La Coruña", "RCD La Coruña")
    df["Teams"] = df["Teams"].str.replace("Milan", "AC Milan")
    df["Teams"] = df["Teams"].str.replace("AC Chievo Verona", "Verona")
    df["Teams"] = df["Teams"].str.replace("Roma", "AS Roma")
    df["Teams"] = df["Teams"].str.replace("Cagliari", "Cagliari Calcio")
    df["Teams"] = df["Teams"].str.replace("Tottenham Hotspur", "Tottenham")
    df["Teams"] = df["Teams"].str.replace("Leicester City", "Leicester")
    df["Teams"] = df["Teams"].str.replace("Leeds United", "Leeds")
    df["Teams"] = df["Teams"].str.replace("West Ham United", "West Ham")
    df["Teams"] = df["Teams"].str.replace("Brighton & Hove Albion", "Brighton")
    df["Teams"] = df["Teams"].str.replace("AS Roma", "Roma")
    df["Teams"] = df["Teams"].str.replace("Parma", "Parma Calcio 1913")
    df["Teams"] = df["Teams"].str.replace("Atlético Madrid", "Atletico Madrid")
    df["Teams"] = df["Teams"].str.replace("FC Barcelona", "Barcelona")
    df["Teams"] = df["Teams"].str.replace("Sevilla FC", "Sevilla")
    df["Teams"] = df["Teams"].str.replace("Granada CF", "Granada")
    df["Teams"] = df["Teams"].str.replace("Cádiz CF", "Cadiz")
    df["Teams"] = df["Teams"].str.replace("SD Eibar", "Eibar")
    df["Teams"] = df["Teams"].str.replace("Valencia CF", "Valencia")
    df["Teams"] = df["Teams"].str.replace("Elche CF", "Elche")
    df["Teams"] = df["Teams"].str.replace("Deportivo Alavés", "Deportivo Alaves")
    df["Teams"] = df["Teams"].str.replace("Getafe CF", "Getafe")
    df["Teams"] = df["Teams"].str.replace("Real Valladolid CF", "Real Valladolid")
    df["Teams"] = df["Teams"].str.replace("Levante UD", "Levante")
    df["Teams"] = df["Teams"].str.replace("CA Osasuna", "Osasuna")
    df["Teams"] = df["Teams"].str.replace("Villarreal CF", "Villarreal")
    # df["Teams"] = df["Teams"].str.replace("RC Celta Vigo", "Celta Vigo")
    df["Teams"] = df["Teams"].str.replace("RC Celta", "Celta Vigo")
    df["Teams"] = df["Teams"].str.replace("Athletic Club de Bilbao", "Athletic Bilbao")
    df["Teams"] = df["Teams"].str.replace("FC Internazionale Milano", "Inter")
    df["Teams"] = df["Teams"].str.replace("Sassuolo Calcio", "Sassuolo")
    df["Teams"] = df["Teams"].str.replace("Chievo Verona", "Verona")
    df["Teams"] = df["Teams"].str.replace("SS Lazio", "Lazio")
    df["Teams"] = df["Teams"].str.replace("Cagliari Calcio", "Cagliari")
    df["Teams"] = df["Teams"].str.replace("ACF Fiorentina", "Fiorentina")
    df["Teams"] = df["Teams"].str.replace("FC Crotone", "Crotone")
    return df

In [54]:
path_2021 = "../raw/fifa21_male2.csv"
players_df_2020 = playersfunc2(path_2021,2020)
df_final2 = df_final.merge(players_df_2020,on=["Teams"])

In [99]:
leaguess_df = leagues_df.merge(df_final2,on=['Teams','Year'],how="left")
leaguess_df.sort_values("Year",ascending=False, inplace=True)
leaguess_df.reset_index(drop=True, inplace=True)
leaguess_df.rename({"Goals+_x":"Goals+","Goals-_x":"Goals-","MA_x":"MA",\
    "Possession%_x":"Possession%","Pass%_x":"Pass%","Shots pg_x":"Shots pg"},axis=1,inplace=True)
leaguess_df["Goals+"].iloc[0:60] = leaguess_df["Goals+_y"].iloc[0:60]
leaguess_df["Goals-"].iloc[0:60] = leaguess_df["Goals-_y"].iloc[0:60]
leaguess_df["MA"].iloc[0:60] = leaguess_df["MA_y"].iloc[0:60]
leaguess_df["Shots pg"].iloc[0:60] = leaguess_df["Shots pg_y"].iloc[0:60]
leaguess_df["avg player rating"].iloc[0:60] = leaguess_df["overall"].iloc[0:60]
leaguess_df["Possession%"].iloc[0:60] = leaguess_df["Possession%_y"].iloc[0:60]
leaguess_df["Pass%"].iloc[0:60] = leaguess_df["Pass%_y"].iloc[0:60]
leaguess_df.drop(["Shots pg_y","Possession%_y","Pass%_y","Goals+_y","Goals-_y","MA_y","overall"],axis=1,inplace=True)
leagues_2020_df = leaguess_df[["Country","Ranking","Teams","Points","Year","Goals+","Goals-",\
    "avg player rating","Shots pg","Possession%","Pass%","MA"]].iloc[0:60,:].\
        sort_values(["Country","Ranking"]).reset_index(drop=True)

In [100]:
final_eng.to_csv('../cleaned_final/epl.csv')
final_es.to_csv('../cleaned_final/liga.csv')
final_it.to_csv('../cleaned_final/serieA.csv')
leaguess_df.to_csv('../cleaned_final/leagues.csv')
leagues_2020_df.to_csv('../cleaned_final/leagues_2020_data.csv')

['Teams', 'Shots pg', 'Possession%', 'Pass%', 'Goals+', 'Goals-', 'MA']